<a href="https://colab.research.google.com/github/LorenzoBellomo/InformationRetrieval/blob/main/notebooks/4_SearchEngineWeaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weaviate as a Search Engine

In [1]:
!pip install -U weaviate-client
import weaviate
import weaviate.classes.config as wc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.1/350.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


In [2]:
import weaviate
from weaviate.classes.query import MetadataQuery
from weaviate.classes.config import Configure, Property, DataType, Tokenization
from weaviate.classes.query import Filter

client = weaviate.connect_to_embedded()

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.26.6/weaviate-v1.26.6-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 660


Let's create a simple collection that has just one field of texts.  

In [ ]:
client.collections.delete_all()
client.collections.create(
    name="TestCollection",
    properties=[
        wc.Property(name="text", data_type=wc.DataType.TEXT),
    ]
)

Here is a list of simple documents that are useful to test some simple queries

In [ ]:
sample_docs = [
    {"text": "Trump u.s.a. NATO"},
    {"text": "trump usa N.A.T.O."},
    {"text": "trump u s a NATO"},
    {"text": "the cat sleeps"},
    {"text": "u are a star"}
]

Now we create the collection and we insert the samples

In [ ]:
documents = client.collections.get("TestCollection")
for doc in sample_docs:
    documents.data.insert(doc)

Here is how to iterate over all documents in the collection

In [ ]:
# retrieve the elements
for doc in documents.iterator():
  print(doc.uuid, " - ", doc.properties)

07112b0b-1083-414c-9c85-a1c86edaaf34  -  {'text': 'the cat sleeps'}
1a5a4245-b2c4-417f-ac00-f97b012e77ea  -  {'text': 'Trump u.s.a. NATO'}
44950b19-9456-47ab-a39b-e2532c13ddf8  -  {'text': 'u are a star'}
915c3213-ac5f-406a-b546-ca2deb259e41  -  {'text': 'trump u s a NATO'}
9b8279cb-7076-4e7b-9812-2e04717f2a07  -  {'text': 'trump usa N.A.T.O.'}


Let's try some simple queries, bm25 is the vectorization textual technique that we saw in lecture 2 (better than TFIDF). This means that the following query is processed textually.

In [ ]:
query = "sleep"
response = documents.query.bm25(query=query, return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["text"]))

Unfortunately, words are not stemmed, but are lowercased. This is on the roadmap of features that Weaviate plans to support in the future.

Let's also define a function that properly prints the results of a query

In [ ]:
def print_query_results(query, prop_name, collection):
  print("QUERY:: {}\n".format(query))
  response = collection.query.bm25(query=query, return_metadata=MetadataQuery(score=True))
  for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties[prop_name]))

In [ ]:
print_query_results("TRUMP", "text", documents) #the words are lowercased

QUERY:: TRUMP

0.24 - trump u s a NATO
0.24 - Trump u.s.a. NATO
0.22 - trump usa N.A.T.O.


In [ ]:
print_query_results("Trump", "text", documents) #the words are lowercased

QUERY:: Trump

0.24 - trump u s a NATO
0.24 - Trump u.s.a. NATO
0.22 - trump usa N.A.T.O.


In [ ]:
print_query_results("the", "text", documents) # the stopwords are not present by assuming English

QUERY:: the



Now we define a function that shows some very basic queries, but that are able

In [ ]:
def example_queries(prop_name, collection):
    queries = ["She is sleeping", "I sleep", "the usa", "I live in the u.s.a.", "TRUMP"]
    for query in queries:
      print_query_results(query, prop_name, collection)
      print("===============================================================")
      print()

In [ ]:
print(sample_docs)
print("\n")
example_queries("text", documents)

[{'text': 'Trump u.s.a. NATO'}, {'text': 'trump usa N.A.T.O.'}, {'text': 'trump u s a NATO'}, {'text': 'the cat sleeps'}, {'text': 'u are a star'}]


QUERY:: She is sleeping


QUERY:: I sleep


QUERY:: the usa

0.56 - trump usa N.A.T.O.

QUERY:: I live in the u.s.a.

0.62 - trump u s a NATO
0.62 - Trump u.s.a. NATO
0.26 - u are a star

QUERY:: TRUMP

0.24 - trump u s a NATO
0.24 - Trump u.s.a. NATO
0.22 - trump usa N.A.T.O.



But how is the input really treated? How is it tokenized?

**TOKENIZATION OPTIONS**
* word: alphanumeric, lowercased tokens (default tokenizer for Weaviate)
* lowercase: lowercased tokens
* whitespace: whitespace-separated, case-sensitive tokens
* the entire value of the property is treated as a single token

In [ ]:
client.collections.create(
    name="TestWhitespace",
    properties=[
        wc.Property(name="text", data_type=wc.DataType.TEXT, tokenization=Tokenization.WHITESPACE),
    ],
)

In [ ]:
documents = client.collections.get("TestWhitespace")
for doc in sample_docs:
    documents.data.insert(doc)

In [ ]:
print_query_results("the", "text", documents) # stopword is found

QUERY:: the

0.68 - the cat sleeps


In [ ]:
print_query_results("Trump", "text", documents) # no lowercasing, thus not find "trump"

QUERY:: Trump

0.68 - Trump u.s.a. NATO


In [ ]:
print_query_results("TRUMP", "text", documents) # no lowercasing, thus not find "trump" and "Trump"

QUERY:: TRUMP



In [ ]:
print_query_results("u", "text", documents) # whitespace does not split "u.s.a." which is one token

QUERY:: u

0.38 - u are a star
0.34 - trump u s a NATO


In [ ]:
print_query_results("u.s.a.", "text", documents)

QUERY:: u.s.a.

0.68 - Trump u.s.a. NATO


In [ ]:
example_queries("text", documents)

QUERY:: She is sleeping


QUERY:: I sleep


QUERY:: the usa

0.68 - the cat sleeps
0.68 - trump usa N.A.T.O.

QUERY:: I live in the u.s.a.

0.68 - the cat sleeps
0.68 - Trump u.s.a. NATO

QUERY:: TRUMP




## Properties
Let's now add some simple properties to our index. As of now we only handled the "text" property, containing some simple textual snippets.

In [8]:
!wget https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/refs/heads/main/data/5articles.json
import json

with open("5articles.json", 'r') as f:
  articles = json.load(f)

--2025-02-26 13:14:39--  https://raw.githubusercontent.com/LorenzoBellomo/InformationRetrieval/refs/heads/main/data/5articles.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12566 (12K) [text/plain]
Saving to: ‘5articles.json’

5articles.json      100%[===================>]  12.27K  --.-KB/s    in 0s      

2025-02-26 13:14:39 (86.5 MB/s) - ‘5articles.json’ saved [12566/12566]



In [ ]:
articles[0]

{'title': 'American Airlines orders 60 Overture supersonic jets',
 'maintext': "The revival of supersonic passenger travel, thought to be long dead with the demise of Concorde nearly two decades ago, could be about to take wing as American Airlines has put in an order for 60 aircraft capable of flying at 1.7 times the speed of sound. \nBoom is a start-up based in Denver, Colorado, whose development of Overture, an ultra-fast successor to Concorde that seats 65 to 88 passengers, is so advanced that it showed off designs at last month's Farnborough air show.",
 'date': '2022-08-18',
 'source': 'The New York Times'}

In [ ]:
client.collections.create(
    name="TestProperties",
    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
    ],
)

In [ ]:
documents = client.collections.get("TestProperties")
for doc in articles:
    documents.data.insert({"maintext": doc["maintext"], "title": doc["title"]})

In [ ]:
for doc in documents.iterator():
  print(doc.uuid, " - ", doc.properties)

256aa134-9e21-4eb9-a695-15496e407f90  -  {'maintext': 'Antonio Conte. Pic: PA\nHead coach Antonio Conte does not think Chelsea are in the race to sign Arsenal forward Alexis Sanchez.\nSanchez is out of contract this summer and seemed certain to join Manchester City this month.\nBut the Premier League leaders on Monday evening decided to end their interest because of the costs involved, with Manchester United in pole position, while there were suggestions the Premier League champions were also in the running.\nConte last Friday spoke of his admiration for Sanchez and described any potential cut-priced deal for the Chile striker as a great opportunity.\nThe Italian was evasive when quizzed on Chelsea\'s interest in the player, taking his usual stance in deferring matters of recruitment to the club.\nAsked if Chelsea were actively seeking to sign Sanchez, Conte said: "I don\'t know. I don\'t think so. I don\'t know, but I don\'t think so."\nConte, speaking ahead of tonight\'s FA Cup third

In [ ]:
print_query_results("mother", "title", documents) # prints the score and the title of the retrieved article

QUERY:: mother

0.52 - 'One-punch killer's sentence will make others think twice'
0.3 - Leclerc dedicates win to Hubert


In [ ]:
print_query_results("cars", "title", documents) # There is no stemming, indeed, thus the next article is not returned

QUERY:: cars

0.48 - Leclerc dedicates win to Hubert


In [ ]:
print_query_results("car", "title", documents) # The score can be larger than 1

QUERY:: car

1.87 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder


Say that you now want to consider some words as "stopwords", that the system does not consider as such by default

In [ ]:
print_query_results("victory", "title", documents) #As above, but below we classify it as a stopword

documents.config.update(inverted_index_config=wc.Reconfigure.inverted_index(stopwords_additions=["victory"]))

print("\n")
print_query_results("victory", "title", documents)

QUERY:: victory

0.71 - Leclerc dedicates win to Hubert


QUERY:: victory



But fields in the query are not all "born equal". Some are more important than others (e.g., title). Let's boost the importance of the "title" field (by scaling its score count by two)

In [ ]:
response = documents.query.bm25(
    query="race",
    return_metadata=MetadataQuery(score=True)
)
print("BEFORE FIELD BOOSTING: (query = race)\n")
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

BEFORE FIELD BOOSTING: (query = race)

1.27 - Conte: 'Chelsea are not in the race to sign Sanchez'
0.54 - Leclerc dedicates win to Hubert


In [ ]:
response = documents.query.bm25(
    query="race",
    query_properties=["title^2", "maintext"],
    return_metadata=MetadataQuery(score=True)
)
print("AFTER FIELD BOOSTING: (query = race)\n")
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

AFTER FIELD BOOSTING: (query = race)

1.43 - Conte: 'Chelsea are not in the race to sign Sanchez'
0.54 - Leclerc dedicates win to Hubert


Add some basic filtering

In [ ]:
response = documents.query.bm25(
    query="mother",
    return_metadata=MetadataQuery(score=True)
)
print("BEFORE FILTERING: (query = mother)\n")
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

BEFORE FILTERING: (query = mother)

0.52 - 'One-punch killer's sentence will make others think twice'
0.3 - Leclerc dedicates win to Hubert


In [ ]:
response = documents.query.bm25(
    query="mother",
    filters=Filter.by_property("title").contains_any(["Leclerc", "formula"]),
    return_metadata=MetadataQuery(score=True)
)
print("AFTER FILTERING: (query = mother)\n")
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

AFTER FILTERING: (query = mother)

0.3 - Leclerc dedicates win to Hubert


Let's see what happens when we also add dates as properties

In [ ]:
client.collections.create(
    name="TestDate",
    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
        wc.Property(name="date", data_type=wc.DataType.DATE)
    ]
)

[All property types](https://weaviate.io/developers/weaviate/config-refs/datatypes)

In [ ]:
from datetime import timezone, datetime
documents = client.collections.get("TestDate")
for doc in articles:
    documents.data.insert({"maintext": doc["maintext"], "title": doc["title"], "date": datetime.strptime(doc["date"], "%Y-%m-%d").replace(tzinfo=timezone.utc)})

In [ ]:
for doc in documents.iterator():
  print(doc.uuid, " - ", doc.properties['date'], '  ', doc.properties['title'])
  # print(doc.uuid, " - ", doc.properties)

752e869d-5019-407a-87c2-d908fe367ac9  -  2019-09-01 00:00:00+00:00    Leclerc dedicates win to Hubert
818fbf8d-9d5e-4cb0-91f4-9f9021c6e7d7  -  2018-01-23 00:00:00+00:00    Conte: 'Chelsea are not in the race to sign Sanchez'
832b788e-c074-4d5f-963a-23a4c5c20caf  -  2019-06-29 00:00:00+00:00    'One-punch killer's sentence will make others think twice'
8c39369a-b5ea-465a-a0d5-108d30daffac  -  2022-08-18 00:00:00+00:00    American Airlines orders 60 Overture supersonic jets
adc6e524-43f5-4c94-9ee4-552a45ad3ac3  -  2019-06-07 00:00:00+00:00    Gunman opens fire on car just metres from scene of Hamid Sanambar murder


In [ ]:
response = documents.query.bm25(
    query="mother",
    return_metadata=MetadataQuery(score=True)
)
print("BEFORE FILTERING: (query = mother)\n")
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

BEFORE FILTERING: (query = mother)

0.52 - 'One-punch killer's sentence will make others think twice'
0.3 - Leclerc dedicates win to Hubert


In [ ]:
response = documents.query.bm25(
    query="mother",
    filters=Filter.by_property("date").greater_or_equal(datetime.strptime("2019-08-15", "%Y-%m-%d").replace(tzinfo=timezone.utc)),
    return_metadata=MetadataQuery(score=True)
)
print("AFTER FILTERING: (query = mother)\n")
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

AFTER FILTERING: (query = mother)

0.3 - Leclerc dedicates win to Hubert


Some advanced features

In [ ]:
response = documents.query.bm25(query="sport", return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

In [ ]:
# Unfortunately, we cannot use all the vectorizer modules that are present in Weaviate. Here is a list of the ones that are available
client.get_meta()

{'hostname': 'http://127.0.0.1:8079',
 'modules': {'generative-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'Generative Search - OpenAI'},
  'qna-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'OpenAI Question & Answering Module'},
  'ref2vec-centroid': {},
  'reranker-cohere': {'documentationHref': 'https://txt.cohere.com/rerank/',
   'name': 'Reranker - Cohere'},
  'text2vec-cohere': {'documentationHref': 'https://docs.cohere.ai/embedding-wiki/',
   'name': 'Cohere Module'},
  'text2vec-huggingface': {'documentationHref': 'https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task',
   'name': 'Hugging Face Module'},
  'text2vec-openai': {'documentationHref': 'https://platform.openai.com/docs/guides/embeddings/what-are-embeddings',
   'name': 'OpenAI Module'}},
 'version': '1.26.6'}

Let's use COHERE as a textual vectorizer [https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys). As we can see, using colab we have only a few options for vectorization (openai, cohere, huggingface). Additionally, only one generation model is available (openai).
Cohere provides free sample apis. OpenAI does not.

In [4]:
## You need first to create a KEY !!!!
from google.colab import userdata

client.close()
cohere_key = userdata.get('COHERE_KEY') # MAKE SURE YOU CREATED A KEY
headers = {
    "X-Cohere-Api-Key": cohere_key,
}
client = weaviate.connect_to_embedded(headers=headers)

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 809


HERE TO CHECK HOW TO INTEGRATE MODELS [https://weaviate.io/developers/weaviate/model-providers](https://weaviate.io/developers/weaviate/model-providers)

Now we create the example collection. Please note that we set here the vectorizer (cohere) and the generation module for an experiment that we will do later (openai, only availabe on the paid model).

In [ ]:
client.collections.delete_all()
client.collections.create(
    name="TestVectorizer",
    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            name="maintext_vector",
            source_properties=["maintext"],
            #model="embed-multilingual-light-v3.0"
        )
    ]
)

In [ ]:
documents = client.collections.get("TestVectorizer")
for doc in articles:
    documents.data.insert({"maintext": doc["maintext"], "title": doc["title"]}) # here weaviate performs the vectorization

In [ ]:
print("pure syntactical search: 'sport'\n")
response = documents.query.bm25(query="sport", return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

pure syntactical search: 'sport'



In [ ]:
print("pure vector search: 'sport'\n")
# NOTE THAT WE ALSO NEED THE PARAMETER DISTANCE
response = documents.query.near_text(query="sport", return_metadata=MetadataQuery(score=True, distance=True), limit=3)
for o in response.objects:
  print("{} - {}".format(round(o.metadata.distance*100)/100, o.properties["title"]))

pure vector search: 'sport'

0.61 - Leclerc dedicates win to Hubert
0.61 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder
0.65 - Conte: 'Chelsea are not in the race to sign Sanchez'


In [ ]:
print("pure syntactical search: 'race'\n")
response = documents.query.bm25(query="race", return_metadata=MetadataQuery(score=True))
for o in response.objects:
    print("{} - {}".format(round(o.metadata.score*100)/100, o.properties["title"]))

pure syntactical search: 'race'

1.27 - Conte: 'Chelsea are not in the race to sign Sanchez'
0.54 - Leclerc dedicates win to Hubert


In [ ]:
print("pure vector search: 'race'\n")
# NOTE THAT WE ALSO NEED THE PARAMETER DISTANCE
response = documents.query.near_text(query="race", return_metadata=MetadataQuery(score=True, distance=True), limit=3)
for o in response.objects:
  print("{} - {}".format(round(o.metadata.distance*100)/100, o.properties["title"]))

pure vector search: 'race'

0.61 - Leclerc dedicates win to Hubert
0.61 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder
0.69 - Conte: 'Chelsea are not in the race to sign Sanchez'


In [ ]:
print("hybrid search: 'race'")
response = documents.query.hybrid(query="race", alpha=0.5, return_metadata=MetadataQuery(score=True, explain_score=True), limit=3)
for o in response.objects:
  print("{} - {} [{}]".format(round(o.metadata.score*100)/100, o.properties["title"],  o.metadata.explain_score.strip().replace("\n", '')))

hybrid search: 'race'
0.59 - Conte: 'Chelsea are not in the race to sign Sanchez' [Hybrid (Result Set keyword,bm25) Document 3a762156-3012-4b82-8b6e-cba4429acd13: original score 1.2714014, normalized score: 0.5 - Hybrid (Result Set vector,hybridVector) Document 3a762156-3012-4b82-8b6e-cba4429acd13: original score 0.31190115, normalized score: 0.09428963]
0.5 - Leclerc dedicates win to Hubert [Hybrid (Result Set keyword,bm25) Document 9f085d01-5641-436d-9b14-c100c75050c2: original score 0.5364737, normalized score: 0 - Hybrid (Result Set vector,hybridVector) Document 9f085d01-5641-436d-9b14-c100c75050c2: original score 0.39159995, normalized score: 0.5]
0.48 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder [Hybrid (Result Set vector,hybridVector) Document 4f12f1fb-5e5b-48fe-b9ed-d59ba14d2ac0: original score 0.38678962, normalized score: 0.47551277]


## A new method, RAG
RAG stands for Retrieval Augmented Generation. This is a recent trend in Information Retrieval that aims at reducing the problem of "hallucinations" for Large Language Model generation.
- Traditional queries go as follows: the user makes a query to the system; the system returns, in some predefined format, the answer to that query.
- LLM queries: the user makes a query to a Large Language Model; the large language model creates an answer based on the (often unspecified) training data that was originally used to train it. The LLM often hallucinates answers.
- RAG: the user makes a query to the system; the system runs the query and gets its result. Before returning the results to the user, they are sent to a LLM to "process" and generate a textual response that is more convenient to read for the user, but (ideally) does not contain hallucinated information because they use precomputed (retrieved) results.

Now let's try to include some generative AI prompts to this query (let's add context to the entities in the news, or let's translate them in Italian).
Note that this query will only work for who has an openai paid module.

In [5]:
client.close()
cohere_key = userdata.get('COHERE_KEY')
openai_key = userdata.get("OPENAI_KEY2")
headers = {
    "X-Cohere-Api-Key": cohere_key,
    "X-OpenAI-Api-Key": openai_key
}
client = weaviate.connect_to_embedded(headers=headers)

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 847


In [11]:
client.collections.delete_all()
client.collections.create(
    name="TestVectorizer",
    properties=[
        wc.Property(name="maintext", data_type=wc.DataType.TEXT, tokenization=Tokenization.WORD),
        wc.Property(name="title", data_type=wc.DataType.TEXT, tokenization=Tokenization.LOWERCASE),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            name="maintext_vector",
            source_properties=["maintext"],
            #model="embed-multilingual-light-v3.0"
        )
    ],
    generative_config=Configure.Generative.openai(model="gpt-4")
)

In [12]:
documents = client.collections.get("TestVectorizer")
for doc in articles:
    documents.data.insert({"maintext": doc["maintext"], "title": doc["title"]}) # here weaviate performs the vectorization

In [20]:
response = documents.generate.near_text(
    query="sport",  # The model provider integration will automatically vectorize the query
    single_prompt="Write a short summary of maximum 100 characters in Italian of {maintext}",
    limit=2
)

In [21]:
for obj in response.objects:
    print(obj.properties["title"])
    print(f"Generated output: {obj.generated}")  # Note that the generated output is per object
    print("====================================================")
    print()

Leclerc dedicates win to Hubert
Generated output: Charles Leclerc ha ottenuto la sua prima vittoria in Formula Uno al Gran Premio del Belgio.

Gunman opens fire on car just metres from scene of Hamid Sanambar murder
Generated output: La polizia irlandese cerca un uomo armato che ha sparato a una macchina a Dublino, vicino al luogo dove Hamid Sanambar è stato ucciso.



This is just to use RAG with the internal Weaviate module. We can also do a sort of "RAG" in house. We run the query, and we send it to a LLM. Let's again use cohere, which allows us to make a few queries on the free program.

In [ ]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.0 MB/s eta 0:00:00


In [ ]:
print("hybrid search: 'race'")
response = documents.query.hybrid(query="race", alpha=0.5, return_metadata=MetadataQuery(score=True, explain_score=True), limit=3)
for o in response.objects:
  print("{} - {} [{}]".format(round(o.metadata.score*100)/100, o.properties["title"],  o.metadata.explain_score.strip().replace("\n", '')))

hybrid search: 'race'
0.59 - Conte: 'Chelsea are not in the race to sign Sanchez' [Hybrid (Result Set keyword,bm25) Document 668ecf7e-757c-42e0-9881-945fb5e8bfbd: original score 1.2714014, normalized score: 0.5 - Hybrid (Result Set vector,hybridVector) Document 668ecf7e-757c-42e0-9881-945fb5e8bfbd: original score 0.31191504, normalized score: 0.094415955]
0.5 - Leclerc dedicates win to Hubert [Hybrid (Result Set keyword,bm25) Document a1d5673b-5dd6-4dbc-806d-2f52889d83e9: original score 0.5364737, normalized score: 0 - Hybrid (Result Set vector,hybridVector) Document a1d5673b-5dd6-4dbc-806d-2f52889d83e9: original score 0.39161122, normalized score: 0.5]
0.48 - Gunman opens fire on car just metres from scene of Hamid Sanambar murder [Hybrid (Result Set vector,hybridVector) Document 6d196d4d-a4f0-494e-b377-475db996731f: original score 0.3867746, normalized score: 0.47538581]


In [ ]:
import cohere

co = cohere.ClientV2(api_key=cohere_key)
res = co.chat(
    model="command-r-plus-08-2024",
    messages=[
        {
            "role": "user",
            "content": "Write a short summary, in Italian of the textual article provided below: \n\n {}".format(response.objects[0].properties["maintext"]),
        }
    ],
)

print(res.message.content[0].text)

L'allenatore del Chelsea, Antonio Conte, ha dichiarato di non credere che il club sia in corsa per l'acquisto dell'attaccante dell'Arsenal, Alexis Sanchez, il cui contratto scade in estate. Il Manchester City, inizialmente interessato, ha abbandonato la trattativa per i costi elevati, lasciando il Manchester United in pole position. Conte ha espresso ammirazione per Sanchez, ma ha evitato di commentare l'interesse del Chelsea, rimandando le decisioni di mercato al club. Daniel Farke, allenatore del Norwich, si prepara ad affrontare il Chelsea in una partita di FA Cup, sperando in un risultato a sorpresa con un piano di gioco speciale. Farke è consapevole della forza del Chelsea, ma è fiducioso che il Norwich possa avere le sue chance se giocherà con coraggio e manterrà il possesso palla.
